In [73]:
import sys
import os
import math
import random
import re
import binascii
import itertools
from pyspark.rdd import RDD
from pyspark import SparkConf, SparkContext
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [74]:
K_SHINGLE = 3
NUM_DOCUMENTS = 101
NUM_SIGNATURE = 100
NUM_BAND = 50
NUM_ROW = int(NUM_SIGNATURE/NUM_BAND)
NEXT_PRIME = 4294967311 # A prime larger lan 2^32-1

In [75]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("LSH")
sc = SparkContext(conf=conf)

In [76]:
docs = sc.wholeTextFiles(os.path.join('data', 'athletics'))

## Split Document

In [77]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n","\n<stop>")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".","<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    text = text.lower()
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    sentences= list(filter(None, sentences))
    return sentences

def split_document_into_words(doc):
    sentences = split_into_sentences(doc)
    return re.findall(r'[^\s!,?":;]+', ' '.join(sentences))

## Shingling

### get_filename
將檔案的路徑切割轉換後得到filename

### split_document_into_words
將document經過一些處理去除非必要的標點符號與並統一轉換成小寫後以字為單位做切割

### shingling
將word lists以K為三做shingling並將得到trigram相接成串透過binascii.crc32 hash到大小為2^32次方的bucket中得到一個hash value

In [78]:
def get_filename(x):
    path = x[0]
    path = os.path.basename(path)
    path = os.path.splitext(path)
    return (int(path[0]), x[1])

def shingling(words):
    shingles = []
    for i in range(len(words)-K_SHINGLE+1):
        shingle = f"{words[i]} {words[i+1]} {words[i+2]}"
        shingles.append(binascii.crc32(shingle.encode("utf-8")) & 0xffffffff)
    return shingles

In [79]:
shingles = docs.map(get_filename) \
            .mapValues(split_document_into_words) \
            .mapValues(shingling)

## Min Hash

### get_signature
對shingle做min hash得到100個signature

In [80]:
A = random.sample(range(1,2**32-1), NUM_SIGNATURE)
B = random.sample(range(1,2**32-1), NUM_SIGNATURE)

def min_hash(a, b, shingles):
    hashes = [((a * x) + b) % NEXT_PRIME for x in shingles]
    return min(hashes)

def get_signature(x):
    doc, shingles = x
    signature = [ min_hash(a, b, shingles) for a,b in zip(A,B) ]
    return (doc, signature)

In [81]:
signatures = shingles.map(get_signature)

## Locality-Sensitive Hashing

### hash_bands
將100個signature以每個row兩個signature hash到50個band中

### candidate_partition
將大於兩個element的candidate pair拆成兩兩一對並保留其signatures
<br>
`
ex. ([signature], (1, 2, 3)) -> ([signature], (1, 2)), ([signature], (1, 3)), ([signature], (2, 3))
`
### reduceBykey(lambda a, b: a + b)
將位於同一個bucket的document合併在一起

In [82]:
def chunk(signatures):
    for i in range(0, len(signatures), NUM_ROW):
        yield frozenset(signatures[i:i + NUM_ROW])
        
def hash_bands(x):
    doc, signatures = x
    bands = [ ((i, hash(b)), [doc]) for i,b in enumerate(chunk(signatures)) ]
    return bands

def candidate_partition(x):
    if len(x) > 2:
        return [list(candidate) for candidate in itertools.combinations(x, 2)]
    else: 
        return [x]

In [83]:
candidates = signatures.flatMap(hash_bands) \
                .reduceByKey(lambda a, b: a + b) \
                .filter(lambda v: len(v[1]) > 1) \
                .mapValues(candidate_partition) \
                .flatMapValues(lambda x: x) \
                .map(lambda x: (tuple(x[1]), x[0][0])) \
                .groupByKey()

## Jaccard Simularity

### jaccard_simularity
根據得到的candidate pair利用signature計算其jaccard simularity

In [84]:
def jaccard_simularity(pairs):
    p1, p2 = pairs
    (c1, signature1) = p1
    (c2, signature2) = p2
    simularity = len(set(signature1).intersection(set(signature2))) / NUM_SIGNATURE
    return ((c1, c2), simularity)

In [85]:
similarities = candidates.map(lambda x: (x[0][1], x[0][0])).join(signatures)\
                    .map(lambda x: (x[1][0], (x[0], x[1][1]))).join(signatures)\
                    .map(lambda x: ((x[0], x[1][1]), x[1][0])) \
                    .map(jaccard_simularity) \
                    .top(10, key=lambda x: x[1])

## Output top10

In [86]:
for s in similarities:
    (c1, c2) = s[0]
    if c1 > c2:
        print(f"({c2}, {c1}): {s[1]*100:.1f}%")
    else:
        print(f"({c1}, {c2}): {s[1]*100:.1f}%")

(12, 20): 100.0%
(52, 84): 100.0%
(30, 35): 78.0%
(47, 49): 71.0%
(48, 49): 54.0%
(49, 88): 44.0%
(23, 38): 41.0%
(14, 40): 38.0%
(47, 48): 37.0%
(47, 88): 33.0%
